In [2]:
import io
import itertools

import numpy as np
import sklearn.metrics
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

In [9]:
#loading the data
train_data = np.load(r'datasets/Glasses & Sunglasses - Train.npz')
val_data = np.load(r'datasets/Glasses & Sunglasses - Validation.npz')
test_data = np.load(r'datasets/Glasses & Sunglasses - Test.npz')

In [10]:
train_images, train_labels = train_data['images'], train_data['labels']
val_images, val_labels = val_data['images'], val_data['labels']
test_images, test_labels = test_data['images'], test_data['labels']

In [11]:
#feature scaling
train_images = train_images/255.
val_images = val_images/255.
test_images = test_images/255.

In [12]:
#constant definition
NUM_EPOCHS = 15
BATCH_SIZE = 64

In [13]:
#hyperparam definition
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([3,5,7]))
HP_NUM_FILTER = hp.HParam('filters_number', hp.Discrete([32,64,96,128]))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer(r'Logs/Model 2/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams= [HP_FILTER_SIZE,HP_NUM_FILTER],
        metrics = [hp.Metric(METRIC_ACCURACY,display_name='accuracy')]
    )

In [14]:
#model definition and training
def train_test_model(hparams,session_num):
    
    #model outline
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_NUM_FILTER],hparams[HP_FILTER_SIZE], activation='relu', input_shape=(120,90,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_NUM_FILTER],3,activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(2)
    ])
    
    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    
    model.compile(optimizer='adam', loss= loss_fn, metrics=['accuracy'])
    
    #defining the logging dir
    log_dir = 'Logs\\Model 2\\fit\\'+'run-{}'.format(session_num)
    
    def plot_confusion_matrix(cm, class_names):
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label') 
        return figure

    def plot_to_image(figure):
    
        #save the plot to a png in memory
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
    
        #closing the figure to prevent from displaying in the notebook
        plt.close(figure)
    
        buf.seek(0)
    
        image = tf.image.decode_png(buf.getvalue(), channels=4)
    
        #add the batch dimension
        image = tf.expand_dims(image, 0)
    
        return image

    #file writer for logging cofusion matrix
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')
    
    def log_confusion_matrix(epoch, logs):
    #predict values for validation set
        test_pred_r = model.predict(val_images)
        test_pred = np.argmax(test_pred_r, axis=1)
    
        cm = sklearn.metrics.confusion_matrix(val_labels, test_pred)
    
        figure = plot_confusion_matrix(cm, class_names=['Glasses', 'Sunglasses'])
        cm_image = plot_to_image(figure)
    
        #log the confusion matrix as image summary
        with file_writer_cm.as_default():
            tf.summary.image('Confusion Matrix', cm_image, step=epoch)
            
    
    
    #defining callbacks
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq = 1, profile_batch = 0)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end= log_confusion_matrix)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        mode = 'auto',
        verbose=0,
        patience = 2,
        min_delta = 0,
        restore_best_weights= True
    )

    
    #training the model
    model.fit(
        train_images,
        train_labels,
        validation_data= (val_images,val_labels),
        verbose=2,
        epochs= NUM_EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks=[tensorboard_callback,cm_callback,early_stopping]
        
    )
    
    #model performance evaluation
    _, accuracy = model.evaluate(val_images,val_labels)
    
    #saving the model
    model.save(r'saved Models\Model 2\Run-{}'.format(session_num))
    
    return accuracy


In [15]:
#logging the result function
def run(log_dir, hparams, session_num):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)


In [16]:
session_num = 1

for filter_size in HP_FILTER_SIZE.domain.values:
    for num_filter in HP_NUM_FILTER.domain.values:
        
        hparams = {
            HP_FILTER_SIZE : filter_size,
            HP_NUM_FILTER : num_filter
        }
        
        run_name = 'run-%d' %session_num
        print('---- Starting trial : %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('Logs/Model 2/hparam_tuning/' +run_name, hparams, session_num)
        
        session_num += 1

---- Starting trial : run-1
{'filter_size': 3, 'filters_number': 32}
Epoch 1/15
63/63 - 17s - loss: 0.4452 - accuracy: 0.7814 - val_loss: 0.2100 - val_accuracy: 0.9260
Epoch 2/15
63/63 - 18s - loss: 0.1615 - accuracy: 0.9475 - val_loss: 0.2326 - val_accuracy: 0.9240
Epoch 3/15
63/63 - 18s - loss: 0.1337 - accuracy: 0.9595 - val_loss: 0.2044 - val_accuracy: 0.9560
Epoch 4/15
63/63 - 18s - loss: 0.1267 - accuracy: 0.9618 - val_loss: 0.2003 - val_accuracy: 0.9520
Epoch 5/15
63/63 - 19s - loss: 0.1276 - accuracy: 0.9615 - val_loss: 0.2086 - val_accuracy: 0.9600
Epoch 6/15
63/63 - 18s - loss: 0.1297 - accuracy: 0.9640 - val_loss: 0.1984 - val_accuracy: 0.9520
Epoch 7/15
63/63 - 18s - loss: 0.1210 - accuracy: 0.9658 - val_loss: 0.1923 - val_accuracy: 0.9560
Epoch 8/15
63/63 - 18s - loss: 0.1179 - accuracy: 0.9673 - val_loss: 0.2050 - val_accuracy: 0.9540
Epoch 9/15
63/63 - 18s - loss: 0.1152 - accuracy: 0.9668 - val_loss: 0.1844 - val_accuracy: 0.9600
Epoch 10/15
63/63 - 18s - loss: 0.1113 -

Epoch 5/15
63/63 - 26s - loss: 0.1522 - accuracy: 0.9540 - val_loss: 0.1932 - val_accuracy: 0.9580
Epoch 6/15
63/63 - 26s - loss: 0.1324 - accuracy: 0.9583 - val_loss: 0.2088 - val_accuracy: 0.9600
Epoch 7/15
63/63 - 26s - loss: 0.1368 - accuracy: 0.9613 - val_loss: 0.2183 - val_accuracy: 0.9600
16/16 [==============================] - 1s 51ms/step - loss: 0.1932 - accuracy: 0.9580
INFO:tensorflow:Assets written to: saved Models\Model 2\Run-9\assets
---- Starting trial : run-10
{'filter_size': 7, 'filters_number': 64}
Epoch 1/15
63/63 - 51s - loss: 0.5002 - accuracy: 0.7634 - val_loss: 0.2402 - val_accuracy: 0.9160
Epoch 2/15
63/63 - 53s - loss: 0.1615 - accuracy: 0.9523 - val_loss: 0.2660 - val_accuracy: 0.9140
Epoch 3/15
63/63 - 49s - loss: 0.1522 - accuracy: 0.9520 - val_loss: 0.2151 - val_accuracy: 0.9600
Epoch 4/15
63/63 - 47s - loss: 0.1483 - accuracy: 0.9580 - val_loss: 0.2053 - val_accuracy: 0.9460
Epoch 5/15
63/63 - 48s - loss: 0.1498 - accuracy: 0.9540 - val_loss: 0.1932 - va

In [19]:
%load_ext tensorboard
%tensorboard --logdir "Logs/Model 2/hparam_tuning"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [18]:
%load_ext tensorboard
%tensorboard --logdir "Logs/Model 2/fit"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
